In [9]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os

class JobinjaScraper:
    def __init__(self, base_url, headers):
        self.base_url = base_url
        self.headers = headers
        self.subpage_links = set()
        self.all_jobs_data = []

    def get_links(self):
        try:
            response = requests.get(self.base_url, headers=self.headers, verify=False, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all('a', href=True)

            for link in links:
                href = link['href']
                if href.startswith('/'):  # It's a relative link
                    full_url = self.base_url + href
                elif href.startswith('http'):
                    full_url = href
                else:
                    continue  # Skip if the href doesn't look like a valid link

                self.subpage_links.add(full_url)

        except requests.exceptions.SSLError as e:
            print(f"SSL error occurred: {e}")
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")

    def extract_subpage_text(self):
        for subpage_url in self.subpage_links:
            try:
                subpage_response = requests.get(subpage_url, headers=self.headers, verify=False, timeout=10)
                subpage_response.raise_for_status()

                subpage_soup = BeautifulSoup(subpage_response.text, 'html.parser')
                subpage_text = subpage_soup.get_text(strip=True)
                
                # Extract additional job data for sorting
                job_title = subpage_soup.find('h1', {'class': 'job-title'})
                job_type = subpage_soup.find('div', {'class': 'job-type'})
                job_location = subpage_soup.find('div', {'class': 'job-location'})
                company_name = subpage_soup.find('div', {'class': 'company-name'})
                contract_type = subpage_soup.find('div', {'class': 'contract-type'})
                work_experience = subpage_soup.find('div', {'class': 'work-experience'})
                min_salary = subpage_soup.find('div', {'class': 'min-salary'})
                gender = subpage_soup.find('div', {'class': 'gender'})
                education_level = subpage_soup.find('div', {'class': 'education-level'})

                job_data = {
                    "url": subpage_url,
                    "content_snippet": subpage_text[:1000],
                    "job_title": job_title.get_text(strip=True) if job_title else "N/A",
                    "job_type": job_type.get_text(strip=True) if job_type else "N/A",
                    "job_location": job_location.get_text(strip=True) if job_location else "N/A",
                    "company_name": company_name.get_text(strip=True) if company_name else "N/A",
                    "contract_type": contract_type.get_text(strip=True) if contract_type else "N/A",
                    "work_experience": work_experience.get_text(strip=True) if work_experience else "N/A",
                    "min_salary": min_salary.get_text(strip=True) if min_salary else "N/A",
                    "gender": gender.get_text(strip=True) if gender else "N/A",
                    "education_level": education_level.get_text(strip=True) if education_level else "N/A",
                }
                self.all_jobs_data.append(job_data)

                print(f"URL: {subpage_url}\nContent Snippet: {subpage_text[:1000]}\n")
                time.sleep(1)  # To avoid overwhelming the server

            except requests.exceptions.RequestException as e:
                print(f"An error occurred while accessing {subpage_url}: {e}")

    def scrape(self):
        self.get_links()
        if not self.subpage_links:
            print("No links found to scrape.")
            return
        self.extract_subpage_text()

    def descriptive_statistics(self):
        # Generate descriptive statistics for the dataset
        df = pd.DataFrame(self.all_jobs_data)
        if not df.empty:
            print(df.describe(include='all'))
        else:
            print("No data available to generate statistics.")

    def sort_data(self):
        # Convert all_jobs_data to a DataFrame and sort by multiple columns
        df = pd.DataFrame(self.all_jobs_data)
        if not df.empty:
            df_sorted = df.sort_values(by=['job_title', 'job_type', 'job_location', 'contract_type', 'work_experience', 'min_salary', 'gender', 'education_level'])
            print("Sorted Data:")
            print(df_sorted)

            # Display sorted data as a matrix
            print("Data Matrix:")
            print(df_sorted.to_string(index=False))

            # Save each sorted job data to a .txt file in the specified directory
            save_dir = "C:/Users/negin/jobinja_sorted_data"
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            
            for index, row in df_sorted.iterrows():
                job_title_safe = row['job_title'].replace('/', '_').replace(' ', '_')
                file_path = os.path.join(save_dir, f"job_{index + 1}_{job_title_safe}.txt")
                with open(file_path, 'w', encoding='utf-8') as file:
                    for key, value in row.items():
                        file.write(f"{key.capitalize()}: {value}\n")
        else:
            print("No data available to sort.")

# Example usage
if __name__ == "__main__":
    url = "https://jobinja.ir/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    scraper = JobinjaScraper(url, headers)
    scraper.scrape()
    scraper.descriptive_statistics()
    scraper.sort_data()


C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/jobs?filters%5Bjob_categories%5D%5B%5D=%D8%AA%D8%B1%D8%A8%DB%8C%D8%AA+%D8%A8%D8%AF%D9%86%DB%8C
Content Snippet: استخدام مربی بدنسازی | استخدام کارشناس ورزش و سلامت
 | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترجستجودرخواست‌های مننشان‌شده‌هاآگهی‌های پیشنهادیاز آگهی‌های جدید باخبر شوبهترین فرصت های شغلی در جابینجاهمه‌ی استان‌هاتهرانخراسان رضویاصفهانالبرزفارسقمآذربایجان شرقیمازندرانگیلانخوزستانکرمانیزدقزوینمرکزیهرمزگانگلستانزنجانآذربایجان غربیبوشهرسیستان و بلوچستانسمنانکرمانشاههمداناردبیللرستانکردستانخراسان شمالیخراسان جنوبیچهارمحال بختیاریایلامکهکیلویه و بویراحمدهمه‌ی دسته‌بندی‌هاتربیت بدنیفروش و بازاریاب

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/jobs?filters%5Blocations%5D%5B%5D=%D8%AE%D8%B1%D8%A7%D8%B3%D8%A7%D9%86+%D8%B4%D9%85%D8%A7%D9%84%DB%8C
Content Snippet: آگهی استخدام خراسان شمالی | استخدام خراسان شمالی | کاریابی خراسان شمالی
 | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترجستجودرخواست‌های مننشان‌شده‌هاآگهی‌های پیشنهادیاز آگهی‌های جدید باخبر شوبهترین فرصت های شغلی در جابینجاهمه‌ی استان‌هاخراسان شمالیتهرانخراسان رضویاصفهانالبرزفارسقمآذربایجان شرقیمازندرانگیلانخوزستانکرمانیزدقزوینمرکزیهرمزگانگلستانزنجانآذربایجان غربیبوشهرسیستان و بلوچستانسمنانکرمانشاههمداناردبیللرستانکردستانخراسان جنوبیچهارمحال بختیاریایلامکهکیلویه و بویراحمدهمه‌ی دسته‌بندی‌

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

URL: https://jobinja.ir/knowledge/job-seeker-faq/pages/job-seeker-account-faq
Content Snippet: سوالات متداول حساب کاربری کارجویان | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترسوالات متداول کارجویانسوالات متداول حساب کاربری کارجویانسوالات متداول حساب کاربری کارجویانثبت نام  و ورود به پنل کارجویی:چگونه می‌توانم در جابینجا حساب کاربری ایجاد کنم؟برای ایجاد حساب کاربری وارد سایت jobinja.ir شوید و از نوار بالای صفحه روی قسمت ثبت نام کارجو کلیک نمایید. بعد از وارد کردن آدرس ایمیل، نام و نام خانوادگی و رمز عبور روی گزینه ثبت نام کلیک کنید تا حساب کاربری شما ساخته شود.چگونه می‌توانم وارد حساب کاربری خود شوم؟بعد از ورود به سایت jobinja.

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/companies/admoon/jobs
Content Snippet: فرصت‌های شغلی فعال در ادمون | Admoon | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترادمون|Admoonتاسیس در  ۱۳۹۷تبلیغات، بازاریابی و برندسازی۱۱ تا ۵۰ نفرadmoon.net/?job=trueدر یک نگاهمعرفیفرصت‌های شغلی۹معرفی ادمونادمون، پلتفرم ارائه دهنده‌ی سرویس تبلیغات گوگل (Google Ads) با هدف جلوگیری از هدر رفت هزینه تبلیغات آنلاین و پیاده سازی راهکار های نتیجه‌بخش روی کمپین های گوگل وارد این عرصه شد و به پشتوانه تجربه‌ی هفت ساله در تبلیغات ‌‌، نبوغ بالای سرمایه انسانی و طراحی ماژول هایی اثربخش برای کمپین های تبلیغاتی داخلی و خارجی تمایز رو هم در داخل ایران و هم عرصه بین‌المللی با ک

C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\negin\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobinja.ir'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


URL: https://jobinja.ir/jobs?filters%5Blocations%5D%5B%5D=%D8%AE%D9%88%D8%B2%D8%B3%D8%AA%D8%A7%D9%86
Content Snippet: آگهی استخدام خوزستان | استخدام خوزستان | کاریابی خوزستان
 | جابینجاامکان جدید:همین حالا رزومه خود را در کمتر از ۱۰ دقیقه بسازید،برو به رزومه‌سازJobinja | جابینجا | استخدام، آگهی استخدام، کاریابی، استخدامی جدید، بازار کار، نیازمندیها، ساخت رزومه رایگان، کار و استخدام دولتی، اشتغال، جستجوگر حرفه ای استخدام و شغلخانهجستجوی مشاغلرزومه‌ساز۵۰ شرکت برتربخش کارفرمایانورود کارجوثبت‌نام کارجوورود کارجوثبت‌نام کارجو۱جابینجا چطور به استخدام شدن من کمک می‌کند؟مشاهدهمشاهده بیشترجستجودرخواست‌های مننشان‌شده‌هاآگهی‌های پیشنهادیاز آگهی‌های جدید باخبر شوبهترین فرصت های شغلی در جابینجاهمه‌ی استان‌هاخوزستانتهرانخراسان رضویاصفهانالبرزفارسقمآذربایجان شرقیمازندرانگیلانکرمانیزدقزوینمرکزیهرمزگانگلستانزنجانآذربایجان غربیبوشهرسیستان و بلوچستانسمنانکرمانشاههمداناردبیللرستانکردستانخراسان شمالیخراسان جنوبیچهارمحال بختیاریایلامکهکیلویه و بویراحمدهمه‌ی دسته‌بندی‌هافروش و بازاریابیوب،‌ برنامه‌نویسی و نر